In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import sys
sys.path.append('..')

import os
import numpy as np
from argparse import Namespace
import cv2
import torch
from utils.common.setup_helper import lprint

import glob
from PIL import Image
from utils.eval.geometry import abs2relapose, pose2fund
from utils.eval.measure import sampson_distance
from transforms3d.quaternions import mat2quat, quat2mat
from utils.datasets.preprocess import crop_from_bottom_right

args = Namespace(base_dir='../data/MegaDepth_undistort',
                 save_dir='../data_pairs/generated',                
                 min_overlap_ratio=0.35, im_target_ratio=1.5, 
                 max_scene_pairs=500, exclude_tag='excl_test')

base_dir = args.base_dir
scene_info_dir = os.path.join(base_dir, 'scene_info')

# Output dir for log and match_pairs npy
save_dir = args.save_dir
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
log_path = os.path.join(save_dir, 'megadepth_pair.log')
log = open(log_path, 'a')
lprint_ = lambda s: lprint(s, log)
lprint_(str(args))
    
min_overlap_ratio = args.min_overlap_ratio
im_target_ratio = args.im_target_ratio
max_scene_pairs = args.max_scene_pairs  # At most take such amount of pairs per scene

scene_overlappings = []
match_dict = {}  # {'ims', 'pairs'}
total_valid_pairs = 0
selected_pairs = 0
scene_files = sorted(os.listdir(scene_info_dir))

# Exclude phototuorism scenes, incase one wants to evaluate on the image matching challenge
exclude_tag = args.exclude_tag
exclude_sce = []
if exclude_tag == 'excl_test':
    exclude_sce = ['0024', '0021', '0025', '1589',
                  '0019', '0008', '0032', '0063']
elif exclude_tag == 'excl_all': 
    exclude_sce = ['0024', '0021', '0025', '1589',
                   '0019', '0008', '0032', '0063',
                   '0015', '0022']


lprint_('Start process {} scenes ......'.format(len(scene_files)))
for scene_file in scene_files:
    scene_name = scene_file.split('.')[0]
    if scene_name in exclude_sce:
        lprint_(f'Skip scene {scene_name}')
        continue
    
    # Load scene information
    scene_info_path = os.path.join(scene_info_dir, scene_file)
    try:        
        scene_info = dict(np.load(scene_info_path, allow_pickle=True))        
    except Exception as err:
        lprint_('Can not open {}: {}'.format(scene_file, err.args))
        continue

    # Select valid pairs
    overlap_matrix = scene_info['overlap_matrix']
    scale_ratio_matrix = scene_info['scale_ratio_matrix']
    valid =  np.logical_and(
        np.logical_and(
            overlap_matrix >= min_overlap_ratio,
            overlap_matrix < 1
        ),
        scale_ratio_matrix <= np.inf
    )
    valid_pair_ids = np.vstack(np.where(valid))   # 2,N
    valid_num = valid_pair_ids.shape[1]
    total_valid_pairs += valid_num
    
    # Load scene information    
    image_paths = scene_info['image_paths']
    depth_paths = scene_info['depth_paths']
    points3D_id_to_2D = scene_info['points3D_id_to_2D']
    points3D_id_to_ndepth = scene_info['points3D_id_to_ndepth']
    intrinsics = scene_info['intrinsics']
    poses = scene_info['poses']   

    # Iterate over pairs
    imlist = {}
    pairs = []
    selected_ids = np.arange(valid_num)
    np.random.shuffle(selected_ids)
    for pair_idx in selected_ids:
        idx1 = valid_pair_ids[0, pair_idx]
        idx2 = valid_pair_ids[1, pair_idx]
        overlap = overlap_matrix[idx1, idx2]

        # Check image ratios
        K1 = intrinsics[idx1]
        K2 = intrinsics[idx2]  
        w1, h1 = 2 * K1[:2, 2]
        w2, h2 = 2 * K2[:2, 2]                    
        if w1 >= h1 and w2 >= h2:
            # Landscape pairs            
            crop1 = crop_from_bottom_right(w1, h1, target_ratio=im_target_ratio)
            crop2 = crop_from_bottom_right(w2, h2, target_ratio=im_target_ratio)
            if not crop1 or not crop2:
                continue
        else:
            # Ignore portrait and mixed pairs 
            continue            

        # Find common 3D points of the curremt image pair
        common_pts3D = np.array(list(
            points3D_id_to_2D[idx1].keys() &
            points3D_id_to_2D[idx2].keys()
        ))

        # Remove matches with bad depth scale??
        pt_nd1 = np.array([points3D_id_to_ndepth[idx1][pid] for pid in common_pts3D])
        pt_nd2 = np.array([points3D_id_to_ndepth[idx2][pid] for pid in common_pts3D])
        scale_ratio = np.maximum(pt_nd1 / pt_nd2, pt_nd2 / pt_nd1)
        common_pts3D = common_pts3D[np.where(scale_ratio <= np.inf)[0]]

        # Obtain 2D correspondences
        matches = []
        for pid in common_pts3D:
            p1 = points3D_id_to_2D[idx1][pid]
            p2 = points3D_id_to_2D[idx2][pid]
            matches.append((p1[0], p1[1], p2[0], p2[1]))
        matches = np.array(matches)
        
       # Calculate relative poses    
        pose1 = poses[idx1]
        r1 = pose1[:3, :3]
        t1 = pose1[:3, 3]
        c1 = - r1.T.dot(t1)
        q1 = mat2quat(r1)

        pose2 = poses[idx2]
        r2 = pose2[:3, :3]
        t2 = pose2[:3, 3]
        c2 = - r2.T.dot(t2)
        q2 = mat2quat(r2)

        t, q = abs2relapose(c1, c2, q1, q2)
        R = quat2mat(q)

        # Check epipolar errors
        K1 = intrinsics[idx1]
        K2 = intrinsics[idx2]  
        F = pose2fund(K1, K2, R, t)

        pts1 = matches[:, :2]
        pts2 = matches[:, 2:]
        dist  = sampson_distance(pts1, pts2, F)
        if np.mean(dist) > 1:
            print('Inaccurate matches or intrinsics')
            continue        
            
        # Wrap up pair data
        im1_name = image_paths[idx1].replace('Undistorted_SfM/', '')
        im2_name = image_paths[idx2].replace('Undistorted_SfM/', '')
        
        if im1_name not in imlist:
            imlist[im1_name] = Namespace(name=im1_name, crop=crop1)      
        if im2_name not in imlist:
            imlist[im2_name] = Namespace(name=im2_name, crop=crop2)         

        pairs.append(Namespace(im1=im1_name, im2=im2_name, overlap=overlap, 
                               K1=K1, K2=K2, t=t, q=q, R=R, crop1=crop1, crop2=crop2))
        scene_overlappings.append(overlap)
        if len(pairs) >= max_scene_pairs:
            break    
    if len(pairs) > 0:
        selected_pairs += len(pairs)
        scene_dict = dict(ims=list(imlist.values()), pairs=pairs)
        match_dict[scene_name] = scene_dict
    lprint_('Scene:{} ims:{} total valid pairs:{} selected pairs:{}'.format(scene_name, len(imlist), valid_num, len(pairs)))    
    
# Save results    
if exclude_tag:
    exclude_tag = '.' + exclude_tag
match_npy_name = 'megadepth_pairs.ov{}_imrat{}.pair{}{}.npy'.format(min_overlap_ratio, im_target_ratio, max_scene_pairs, exclude_tag)
match_npy_path = os.path.join(save_dir, match_npy_name)
np.save(match_npy_path, match_dict)
lprint_('Save mathces to {}.'.format(match_npy_path))    
lprint_('Finished, total valid pairs:{} used scenes:{} pairs:{} '.format(total_valid_pairs, len(match_dict), selected_pairs))
bins = np.arange(0.15, 1.01, 0.1)
hist = np.histogram(scene_overlappings, bins=bins)[0]
lprint_('Overlapping bins={} hist={}'.format(bins, hist))
log.close()

Namespace(base_dir='../data/MegaDepth_undistort', exclude_tag='excl_test', im_target_ratio=1.5, max_scene_pairs=500, min_overlap_ratio=0.35, save_dir='../data_pairs/generated')
Start process 182 scenes ......
Inaccurate matches or intrinsics
Scene:0000 ims:673 total valid pairs:63928 selected pairs:500
Scene:0001 ims:257 total valid pairs:4341 selected pairs:500
Scene:0002 ims:579 total valid pairs:22516 selected pairs:500
Scene:0003 ims:409 total valid pairs:33786 selected pairs:500
Scene:0004 ims:523 total valid pairs:101061 selected pairs:500
Scene:0005 ims:204 total valid pairs:1132 selected pairs:500
Inaccurate matches or intrinsics
Scene:0007 ims:515 total valid pairs:18447 selected pairs:500
Scene:0012 ims:277 total valid pairs:17296 selected pairs:500
Scene:0013 ims:314 total valid pairs:23885 selected pairs:500
Scene:0015 ims:290 total valid pairs:2294 selected pairs:500
Scene:0016 ims:59 total valid pairs:314 selected pairs:259
Scene:0017 ims:356 total valid pairs:8075 select

Scene:0327 ims:146 total valid pairs:7280 selected pairs:439
Scene:0331 ims:420 total valid pairs:38561 selected pairs:500
Scene:0335 ims:217 total valid pairs:2019 selected pairs:500
Scene:0341 ims:189 total valid pairs:20061 selected pairs:500
Inaccurate matches or intrinsics
Scene:0348 ims:114 total valid pairs:1837 selected pairs:298
Scene:0349 ims:8 total valid pairs:17 selected pairs:6
Scene:0360 ims:182 total valid pairs:2740 selected pairs:500
Scene:0366 ims:108 total valid pairs:5272 selected pairs:500
Inaccurate matches or intrinsics
Inaccurate matches or intrinsics
Scene:0377 ims:382 total valid pairs:4253 selected pairs:500
Scene:0380 ims:320 total valid pairs:7113 selected pairs:500
Scene:0387 ims:74 total valid pairs:443 selected pairs:257
Scene:0389 ims:174 total valid pairs:1900 selected pairs:500
Scene:0394 ims:158 total valid pairs:4276 selected pairs:500
Scene:0402 ims:97 total valid pairs:1018 selected pairs:292
Scene:0406 ims:101 total valid pairs:478 selected pair